In [6]:
!pip install h5py
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/9a/f2/0ad053856debbe90c83de1b4f05915f85fd2146f20faf9daa3b320d36df3/pandas-2.0.3-cp39-cp39-win_amd64.whl.metadata
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ------------ ------------------------- 112.6/341.8 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 341.8/341.8 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.8 MB 8.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.1/10.8 MB 11.3 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/10.8 MB 12.6 MB/s eta 0:00:01
   --------- ------------------------------ 2.5/10.8 MB 13.4 MB/s eta 0:00:01
   ----------- ---------------------

In [7]:
# import h5py
import math
import pandas as pd
from tensorflow.keras.utils import Sequence
import numpy as np

In [9]:
!pip install hdf5plugin

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for hdf5plugin from https://files.pythonhosted.org/packages/93/92/e6f276daa2c9f3be4c37c65fa4ebe1809979612a78d7d6069ecdba286dfb/hdf5plugin-4.1.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.7 MB 960.0 kB/s eta 0:00:04
   -- ------------------------------------- 0.3/3.7 MB 3.4 MB/s eta 0:00:02
   ------ --------------------------------- 0.6/3.7 MB 4.8 MB/s eta 0:00:01
   ---------- ----------------------------- 1.0/3.7 MB 5.8 MB/s eta 0:00:01
   ----------- ---------------------------- 1.0/3.7 MB 5.5 MB/s eta 0:00:01
   ----------- ---------------------------- 1.0/3.7 MB 5.5 MB/s eta 0:00:01
   ----------- ---------------------------- 1.0/3.7 MB 5.5 MB/s eta 0:00:01
   ----------- ---------------------------- 1.0/3.7 MB 5.5 MB/s eta 0:00:01
   ----------- ----------------------

In [14]:
!pip install --upgrade h5py --user

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for h5py from https://files.pythonhosted.org/packages/a0/62/9790f98aa125a035cda91be7a41a46bdc76b26ffdd2ad2d3c5b7f7232946/h5py-3.9.0-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.7 MB 2.2 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.7 MB 2.0 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/2.7 MB 2.1 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.7 MB 2.1 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.7 MB 2.1 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.7 MB 1.5 MB/s eta 0:00:02
   -------- ------------------------------- 0.6/2.7 MB 1.5 MB/s eta 0:00:02
   ---------- ----------------------------- 0.7/2.7 MB 1.

In [15]:
try:
    import h5py
    # Your code that uses h5py can go here
except ImportError as e:
    # Handle the DLL error
    print("Error importing h5py:", e)
    print("Make sure the HDF5 C library is installed and accessible.")
    # Optionally, you can install h5py using pip here or perform other error handling actions.

In [17]:

class ECGSequence(Sequence):
    @classmethod
    def get_train_and_val(cls, path_to_hdf5, hdf5_dset, path_to_csv, batch_size=8, val_split=0.02):
        n_samples = len(pd.read_csv(path_to_csv))
        n_train = math.ceil(n_samples*(1-val_split))
        train_seq = cls(path_to_hdf5, hdf5_dset, path_to_csv, batch_size, end_idx=n_train)
        valid_seq = cls(path_to_hdf5, hdf5_dset, path_to_csv, batch_size, start_idx=n_train)
        return train_seq, valid_seq

    def __init__(self, path_to_hdf5, hdf5_dset, path_to_csv=None, batch_size=8,
                 start_idx=0, end_idx=None):
        if path_to_csv is None:
            self.y = None
        else:
            self.y = pd.read_csv(path_to_csv).values
        # Get tracings
        self.f = h5py.File(path_to_hdf5, "r")
        self.x = self.f[hdf5_dset]
        self.batch_size = batch_size
        if end_idx is None:
            end_idx = len(self.x)
        self.start_idx = start_idx
        self.end_idx = end_idx

    @property
    def n_classes(self):
        return self.y.shape[1]

    def __getitem__(self, idx):
        start = self.start_idx + idx * self.batch_size
        end = min(start + self.batch_size, self.end_idx)
        if self.y is None:
            return np.array(self.x[start:end, :, :])
        else:
            return np.array(self.x[start:end, :, :]), np.array(self.y[start:end])

    def __len__(self):
        return math.ceil((self.end_idx - self.start_idx) / self.batch_size)

    def __del__(self):
        self.f.close()